<a href="https://colab.research.google.com/github/Anum-Ilyas9/Assignment3/blob/main/CodeA3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Discrepancies


**1. Backbone Network**

*  **Paper:** BERT with contextual embeddings.
*   **Implementation:** : Uses BERT (bert-base-uncased) for strong contextual understanding; could benefit from domain-specific tuning.

**2. Classifier**

*   **Paper:** Multi-layer perceptron with dropout and softmax.
*  **Implementation:**  Employs a simple dense layer with sigmoid for binary classification, providing efficiency but limited complexity.

**3. Preprocessing**


*   **Paper:** Advanced tokenization, padding, and handling of special tokens.
*   **Implementation:**Involves text normalization, basic tokenization, and padding to ensure clean, consistent input, with room for advanced feature extraction.

**4. Fine-Tuning**


*  **Paper:** Extensive fine-tuning with hyperparameter optimization.
*   **Implementation:**Fine-tuned for 2 epochs at a fixed learning rate, achieving solid performance, with potential for further optimization.

**5. Evaluation Metrics**

*  **Paper Accuracy:** Reported accuracy of 92.0%.
*   **Implementation Results:**

       Accuracy: 92.85%

      Precision: 92.93%

      Recall: 92.01%

      F1 Score: 92.47%





In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import os
import warnings
import random

warnings.filterwarnings("ignore")

# Download dataset
!pip install kaggle --upgrade --quiet
os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)
with open(os.path.expanduser("~/.kaggle/kaggle.json"), "w") as file:
    file.write('{"username":"YOUR_KAGGLE_USERNAME","key":"YOUR_KAGGLE_API_KEY"}')
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d rmisra/news-headlines-dataset-for-sarcasm-detection -p ./data --unzip

# Load dataset
data = pd.read_json("./data/Sarcasm_Headlines_Dataset_v2.json", lines=True)
data['label'] = data['is_sarcastic']

# Cleaning function
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)  # Remove URLs
    text = re.sub(r"[^a-z\s]", '', text)  # Remove special characters
    text = re.sub(r"\s+", ' ', text).strip()  # Remove extra spaces
    return text

# Apply cleaning
data['cleaned_headline'] = data['headline'].apply(clean_text)

# Tokenization and additional preprocessing steps
def tokenize_text(text, tokenizer):
    return tokenizer(text, padding="max_length", truncation=True, max_length=64)

# Train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['cleaned_headline'], data['label'], test_size=0.2, random_state=42
)

# Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Create Datasets
train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels})
val_dataset = Dataset.from_dict({"text": val_texts, "label": val_labels})

# Tokenize dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=64)

train_dataset = train_dataset.map(tokenize_function)
val_dataset = val_dataset.map(tokenize_function)

# Load model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
print("Starting model fine-tuning...")
trainer.train()

# Evaluate the model
print("Evaluating model...")
metrics = trainer.evaluate()

# Display predictions
predictions, labels, _ = trainer.predict(val_dataset)
predicted_labels = np.argmax(predictions, axis=-1)

results_df = pd.DataFrame({
    'True Labels': val_labels,
    'Predicted Labels': predicted_labels,
    'Text': val_texts
})

# Display random predictions
random_sample = results_df.sample(n=10, random_state=42)  # Select 10 random samples

print("\n Predictions of some headlines:")
for index, row in random_sample.iterrows():
     print(f"Text: {row['Text']}, True Label: {row['True Labels']}, Predicted Label: {row['Predicted Labels']}")

# Display results
print("\nFine-tuning Results:")
print(f"Accuracy: {metrics['eval_accuracy']:.4f}")
print(f"Precision: {metrics['eval_precision']:.4f}")
print(f"Recall: {metrics['eval_recall']:.4f}")
print(f"F1 Score: {metrics['eval_f1']:.4f}")


Dataset URL: https://www.kaggle.com/datasets/rmisra/news-headlines-dataset-for-sarcasm-detection
License(s): Attribution 4.0 International (CC BY 4.0)
 91% 3.00M/3.30M [00:00<00:00, 5.48MB/s]
100% 3.30M/3.30M [00:00<00:00, 5.04MB/s]


Map:   0%|          | 0/22895 [00:00<?, ? examples/s]

Map:   0%|          | 0/5724 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting model fine-tuning...


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.190700,0.304241,0.907407,0.946407,0.854159,0.897920
2,0.179000,0.324602,0.928546,0.929312,0.920117,0.924692


Evaluating model...



 Predictions of some headlines:
Text: bid on your airline seat the next time you fly, True Label: 0, Predicted Label: 0
Text: report adjectives tony snarky used only by media, True Label: 1, Predicted Label: 1
Text: man confidently hits send on worst job application company has ever seen, True Label: 1, Predicted Label: 1
Text: movie studio blows whole budget on bigname gaffer, True Label: 1, Predicted Label: 1
Text: these two little kids are better at soccer than you are at anything, True Label: 0, Predicted Label: 0
Text: the yearold actress in the florida project gives the years best screen performance, True Label: 0, Predicted Label: 0
Text: nuclear warhead thrilled for chance to finally escape north korea, True Label: 1, Predicted Label: 1
Text: want to harm your relationship here are easy ways, True Label: 0, Predicted Label: 0
Text: how weve kept our son from feeling like hes from a broken home, True Label: 0, Predicted Label: 0
Text: steve bannon suggests donald trump met with